In [ ]:
import os, pandas, numpy, systole, systole.reports, bokeh

ibi = []
for f in [f for f in os.listdir () if f[-4:].lower () == '.csv' and f != 'current.csv']:
    df = pandas.read_csv (f, usecols = ['simple', 'regular', 'peaks'], dtype = numpy.float64).dropna ()
    df.attrs['name'] = f[:-4]
    ibi.append (df)
if (len (ibi) < 1):
    raise SystemExit ("No CSV files found.")
ibi = pandas.concat (ibi, keys = [df.attrs['name'] for df in ibi])
ibi.index.names = ["Source", "Index"]
source_indexes = ibi.index.get_level_values ("Source").unique ()

def report_domain (callable):
    tds = pandas.concat ([callable (ibi['regular'][i]) for i in source_indexes], keys = source_indexes)
    tds.index.names = ["Source", "Index"]
    return pandas.pivot_table (tds, index = "Source", values = "Values", columns = "Metric")

def show_plot (callable, cols = 1, sizing_mode = None, width = None, **kwargs):
    ps = [callable (ibi['simple'][i], input_type = "rr_ms", backend = "bokeh", **kwargs) for i in source_indexes]
    for i, p in enumerate (ps):
        p.title.text = source_indexes[i]
    bokeh.plotting.show (bokeh.layouts.gridplot (ps, sizing_mode = sizing_mode, width = width, ncols = cols))

bokeh.io.output_notebook ()

In [ ]:
ibi_artefacts = []
for d in [systole.detection.rr_artefacts (ibi['simple'][i]) for i in source_indexes]:
    ibi_artefacts.append (pandas.DataFrame.from_dict (
        {k : [(d[k] == True).sum ()] for k in ['ectopic', 'short', 'long', 'missed', 'extra']}))
pandas.concat (ibi_artefacts, keys = source_indexes)


In [ ]:
report_domain (systole.hrv.time_domain)

In [ ]:
show_plot (systole.plots.plot_rr, sizing_mode = "scale_width", show_artefacts = False, slider = False, figsize = 200)

In [ ]:
report_domain (systole.hrv.frequency_domain)

In [ ]:
show_plot (systole.plots.plot_frequency, cols = 4, width = 400, figsize = 200)

In [ ]:
report_domain (systole.hrv.nonlinear_domain)

In [ ]:
show_plot (systole.plots.plot_poincare, cols = 4, width = 400, figsize = 400)

In [ ]:
show_plot (systole.plots.plot_ectopic , cols = 4, figsize = 400)
show_plot (systole.plots.plot_shortlong , cols = 4, figsize = 400)